# Asignatura: 05MBID -> MINERÍA DE DATOS

# Actividad 2: Preprocesamiento y calidad de datos

Estudiante: Andrea Carolina Peralta Bravo

Fecha: 27/03/2022

## 1. Introducción
### Objetivo:
Importar el conjunto de datos de la actividad 1, pre procesarlos, hacer un análisis de la calidad de los datos, y mostrar una comparativa entre los datos iniciales y los datos finales, tras el conjunto de pasos que se han realizado. En la etapa de preprocesamiento y limpieza eliminaremos datos innecesarios, manejaremos los missing values, trataremos los datos que no estén en el formato apropiado y los outliers, entre otros.

### Información Preliminar:
Para cumplir con el objetivo, se partirá de lo planteado en la actividad 1, donde la librería SODILIBRO ubicada en la ciudad de Cuenca-Ecuador, nos permite utilizar la base de datos de ventas de la empresa
desde el año 2015 hasta el año 2021 así como también la base de datos del inventario que la librería tiene. El objetivo a lograr con el procesos KDD consiste en:
1. Predecir ventas futuras para planificar las importaciones internacionales de libros así como las compras a proveedores locales

2. Incrementar las ventas identificando patrones de compra de sus clientes (categorías de libros, fechas del año, autores, etc.)

3. Recomendación de libros

    3.1. Identificar los libros que se compran iguales para recomendaciones a otros clientes con similares características

    3.2. Identificar las categorías de libros más compradas por un usuario y recomendarle los libros más vendidos en esa categoría

## 2. Preprocesamiento y limpieza

### 2.1. Importar librerías:
Lo primero que haremos será importar las librerías:
1. Pandas: librería de Python especializada en el manejo y análisis de estructuras de datos.
2. Numpy: librería de Python especializada en el cálculo numérico y el análisis de datos
3. Matplotlib: librería de Python especializada en la creación de gráficos en dos dimensiones.

In [1]:
# import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure
from pandas import ExcelWriter
from pandas import ExcelFile

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

pd.options.mode.chained_assignment = None

### 2.2. Cargar Archivos:
Se cargarán los dos archivos csv correspondientes a las Ventas e Inventario, los cuales se almacenarán en los dataframes df_ventas y df_inventario respectivamente.

In [2]:
df_ventas = pd.read_excel('Ventas.xlsx')
df_inventario = pd.read_excel('Inventario.xlsx')

Se verificará el número de filas y número de columnas que tiene cada uno de los datasets :

In [3]:
print(df_ventas.shape)
print(df_inventario.shape)

(158501, 38)
(83756, 27)


De esta manera sabemos que:
1. El dataset de ventas tiene 158501 filas y 38 columnas
2. El dataset de inventario tiene 83756 filas y 27 columnas

Ahora verificaremos cuales son las columnas correspondientes a variables númericas y cuáles no son numéricas

In [4]:
#Ventas
df_ventas_numeric = df_ventas.select_dtypes(include=[np.number])
ventas_numeric_cols = df_ventas_numeric.columns.values
print("Columnas Númericas Ventas: ",ventas_numeric_cols)

df_ventas_non_numeric = df_ventas.select_dtypes(exclude=[np.number])
ventas_non_numeric_cols = df_ventas_non_numeric.columns.values
print("Columnas no Númericas Ventas: ",ventas_non_numeric_cols)

#Inventario
df_inventario_numeric = df_inventario.select_dtypes(include=[np.number])
inventario_numeric_cols = df_inventario_numeric.columns.values
print("Columnas Númericas Inventario: ",inventario_numeric_cols)

df_inventario_non_numeric = df_inventario.select_dtypes(exclude=[np.number])
inventario_non_numeric_cols = df_inventario_non_numeric.columns.values
print("Columnas no Númericas Inventario: ",inventario_non_numeric_cols)

Columnas Númericas Ventas:  ['Cod.Tipo Cliente' 'Cantidad' 'PrecioU' 'PrecioT' 'PrecioUReal'
 'PrecioTReal' 'Diferencias' 'Descuento' 'IVA' 'Total' 'CostoTotalReal'
 'Utilidad' 'Por Utilidad' 'CostoTotalUC' 'UTILIDADUC' 'PorUtilidadUC']
Columnas no Númericas Ventas:  ['CodVendedor' 'NombreVend' 'CodLínea' 'Desc.Línea' 'CodItem' 'CodAlterno'
 'Descripcion' 'Trans' 'NumDoc' 'CodProvCli' 'NombreProvCli' 'Fecha'
 'CodBodega' 'Cod.Modelo' 'Cod.Detalle' 'Desc.Detalle' 'Cod.Grupo6'
 'Desc.Grupo6' 'Cod.Tipo Cliente.1' 'Provincia' 'Canton' 'Parroquia']
Columnas Númericas Inventario:  ['Prcentaje IVA' 'Precio1' 'Precio2' 'Precio3' 'Precio4' 'Precio5'
 'Precio6' 'Precio7' 'Existencia' 'pesoneto' 'pesobruto']
Columnas no Númericas Inventario:  ['Modulo' 'Linea' 'CodGrupo2' 'Grupo2' 'CodGrupo3' 'Grupo3' 'CodGrupo4'
 'Grupo4' 'CodGrupo5' 'Grupo5' 'CodItem' 'Descripcion' 'IVA' 'Servicio'
 'Unidad' 'Observacion']


In [5]:
df_ventas['CodItem']=df_ventas['CodItem'].astype('string')
df_inventario['CodItem']=df_inventario['CodItem'].astype('string')

### 2.3. Eliminar datos no necesarios 

#### FILAS
La librería SODILIBRO, cuenta no solo con la venta de libros dentro de su actividad comercial, también ofrece elementos de papelería y arriendo de oficinas. Para el objetivo planteado, estos últimos no son necesarios, por lo que procedemos a eliminar las filas que no estén relacionados con la venta de libros.

Para ellos partimos de que los bienes y servicios se encuentran categorizados acorde al primer caracter del código del Item:
1. Si empieza con L o con P estamos hablando de libros.
2. Cualquier otra letra de inicio implicaría otro de los productos o servicios que ofrece la librería.

In [6]:
df_v=df_ventas.drop(df_ventas[df_ventas['CodItem'].str.startswith('A')==True].index)
df_i=df_inventario.drop(df_inventario[(df_inventario['CodItem'].str.startswith('L') | df_inventario['CodItem'].str.startswith('P'))==False].index)
print(df_v.shape)
print(df_i.shape)

(83203, 38)
(79994, 27)


#### COLUMNAS
Para el data set de ventas, las siguientes columnas no aportan para el objetivo que queremos cumplir: CodVendedor, CodLinea, CodAlterno, NumDoc, Cod.Tipo Cliente, CodProvCli, NombreProvCli, CodBodega, Cod.Detalle, Desc.Detalle, Cod.Grupo6, Desc.Grupo6, Cod.Tipo Cliente, Provincia, Canton, Parroquia, Total, PrecioU, PrecioUReal, PrecioTReal, Diferencias, Descuento, IVA, CostoTotalReal, Por Utilidad, CostoTotalUC, UTILIDADUC, PorUtilidadUC

Para el data set de inventario, las siguientes columnas no aportan para el objetivo que queremos cumplir: Modulo, Linea, CodGrupo2, CodGrupo3, CodGrupo5, Grupo5, IVA, Prcentaje IVA, Servicio, Precio1, Precio2, Precio3, Precio4, Precio5, Precio6, Precio7, Existencia, Unidad, Observacion, pesoneto, pesobruto

In [7]:
df_v2=df_v.drop(['CodVendedor', 'CodLínea', 'CodAlterno', 'NumDoc', 'CodProvCli', 'NombreProvCli', 'CodBodega', 'Cod.Detalle', 'Desc.Detalle', 'Cod.Grupo6', 'Desc.Grupo6', 'Cod.Tipo Cliente.1', 'Provincia', 'Canton', 'Parroquia', 'PrecioU', 'PrecioUReal', 'PrecioTReal', 'Diferencias', 'Total','Descuento', 'IVA', 'CostoTotalReal', 'Por Utilidad', 'CostoTotalUC', 'UTILIDADUC', 'PorUtilidadUC'], axis=1)
print("VENTAS: ",df_v2.shape)

df_i2=df_i.drop(['Modulo', 'Linea', 'CodGrupo2', 'CodGrupo3', 'CodGrupo5', 'Grupo5', 'IVA', 'Prcentaje IVA', 'Servicio', 'Precio1', 'Precio2', 'Precio3', 'Precio4', 'Precio5', 'Precio6', 'Precio7', 'Existencia', 'Unidad', 'Observacion', 'pesoneto', 'pesobruto'], axis=1)
print("INVENTARIO: ",df_i2.shape)

VENTAS:  (83203, 11)
INVENTARIO:  (79994, 6)


Una vez eliminados las gilas que no se requieren, obtenemos que:
1. El dataset de ventas tiene 83203 filas válidas y 1 columnas válidas
2. El dataset de inventario tiene 79994 filas válidas y 6 columnas válidas

### 2.4. Missing Values 
Para llevar a cabo la limpieza y preprocesado de los datos, lo que haremos en primer lugar será ver si existen missing values. Para ello comenzaremos obteniendo los porcentajes de missing values que hay en cada columna de ambos datasets

In [8]:
print("PORCENTAJES MISSING VALUES - VENTAS")
for col in df_v2.columns:
    pct_missing = np.mean(df_v2[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

PORCENTAJES MISSING VALUES - VENTAS
NombreVend - 0%
Desc.Línea - 0%
CodItem - 0%
Descripcion - 0%
Trans - 0%
Cod.Tipo Cliente - 39%
Fecha - 0%
Cod.Modelo - 0%
Cantidad - 0%
PrecioT - 0%
Utilidad - 0%


In [9]:
print("PORCENTAJES MISSING VALUES - INVENTARIO")
for col in df_i2.columns:
    pct_missing = np.mean(df_i2[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

PORCENTAJES MISSING VALUES - INVENTARIO
Grupo2 - 9%
Grupo3 - 2%
CodGrupo4 - 0%
Grupo4 - 0%
CodItem - 0%
Descripcion - 0%


#### Limpieza de los missing  values

#### A. Eliminar instancias

Para nuestro caso, esto no aplicaría, ya que en el caso del dataset de ventas, el único campo con missing values corresponden al tipo de clientes. Luego de conversar con el personal de la librería, pudimos deteminar que estos clientes que no tienen ningún valor corresponden a clientes frecuentes a los que se les hace un descuento adicional, por lo que más adelante se podrá completar la información.

En el caso del dataset de inventario, los missing values corresponden en porcentaje mínimo  a libro que no tienen autor y/o editorial. Por lo que de igual manera, se completará la información más adelante.

#### B. Reemplazar valores
#### Ventas 
- En este caso tenemos la variable categoríca que corresponde al tipo de clientes. Los clientes regulares tienen un valor de 1, por lo que a los clientes con descuento, que se encuentran en blanco reemplazaremos con el valor de 0.

In [10]:
df_v2['Cod.Tipo Cliente'] = df_v2['Cod.Tipo Cliente'].fillna(0)
df_v2.head()

,NombreVend,Desc.Línea,CodItem,Descripcion,Trans,Cod.Tipo Cliente,Fecha,Cod.Modelo,Cantidad,PrecioT,Utilidad
0,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ...",FC 21353,1.0,27/03/2015,P11,-1,51.00,36.05
1,GLENDA SANCHEZ,ENCICLOPEDIAS,P140149,"GRAN LIBRO DE LA HERRERIA, EL",FC 21208,0.0,19/03/2015,P14,-1,35.10,6.40
2,GLENDA SANCHEZ,ENCICLOPEDIAS,P140148,"GRAN LIBRO DE LAS PUERTAS, EL clásico, moderno...",FC 21077,1.0,12/03/2015,P14,-1,22.75,17.45
3,CLAUDIA BRAVO O.,ENCICLOPEDIAS,P190078,LO MEJOR DE LA COCINA ECUATORIANA (BRAL´S),FC 21640,1.0,14/04/2015,P19,-1,18.00,7.30
4,CLAUDIA BRAVO O.,ENCICLOPEDIAS,P170110,TRATADO DE ESQUIZOFRENIA,FC 21016,1.0,10/03/2015,P17,-1,37.80,15.80


#### Inventario 
- En este caso tenemos la variables correspondientes al autor y a la editorial de las que no se conoce los valores para ciertas tuplas. Lo que haremos en estos casos, es rellenar a estas tuplas con el valor de "desconocido"

In [11]:
df_i2['Grupo2'] = df_i2['Grupo2'].fillna("DESCONOCIDO")
df_i2['Grupo3'] = df_i2['Grupo3'].fillna("DESCONOCIDO")
df_i2[(df_i2.Grupo2 == "DESCONOCIDO") | (df_i2.Grupo3 == "DESCONOCIDO")]

,Grupo2,Grupo3,CodGrupo4,Grupo4,CodItem,Descripcion
226,DESCONOCIDO,ABYA - YALA,L04,Educación - Sociología,L043068,BUEN VIVIR SUMAK KAWSAY
227,DESCONOCIDO,ABYA - YALA,L04,Educación - Sociología,L043068,BUEN VIVIR SUMAK KAWSAY
248,DESCONOCIDO,ABYA - YALA,L04,Educación - Sociología,L043081,TRANSFORMACION ANTROPOLOGICA DEL SIGLO XXI
249,DESCONOCIDO,ABYA - YALA,L04,Educación - Sociología,L043081,TRANSFORMACION ANTROPOLOGICA DEL SIGLO XXI
278,VARIOS AUTORES,DESCONOCIDO,L04,Educación - Sociología,L043102,"CORREISMO AL DESNUDO, EL"
...,...,...,...,...,...,...
79505,"LARA, JAIME",DESCONOCIDO,L04,Educación - Sociología,L042606,JUEGOS DE OTROS TIEMPOS T1
79506,"LARA, JAIME",DESCONOCIDO,L04,Educación - Sociología,L042606,JUEGOS DE OTROS TIEMPOS T1
79509,DESCONOCIDO,TURNER,L04,Educación - Sociología,L042609,POST IT CITY ciudades ocasionales-cidades ocas...
79845,"SARMIENTO, VICENTE",DESCONOCIDO,L04,Educación - Sociología,L042790,ORGANIZADORES GRAFICOS motivaciones para enseñ...


### 2.5. Outliers

Dada la naturaleza de los datos, no se tienen outliers ni en valores categorícas ni en numéricas. Podríamos analizar el tema del Precio Total o de la Utilidad, pero aquí podemos  encontrar valores de $1 hasta valores de más de $100, y estos valores nos permitirán analizar más adelante si el precio puede considerarse o no un factor para la venta de un libro.


### 2.6. Datos innecesario
Al inicio, cuando se realizó la carga del dataset, se tuvieron ya que eliminar datos puesto que los dataset era extremadamente grandes y no se veía conveniente utilizar recursos de procesamiento computacional analizando variables que no aportaban nada, sin embargo ahora podemos darnos cuenta que todavía existen datos repetidos en el dataset de inventario. Esto debido a que se manejan varias bodegas para ubicar a los libros dentro de la librería por lo procederemos a eliminar los libros que se encuentran duplicados tomando como base el código del ítem.

In [12]:
df_i3 = df_i2.drop_duplicates(subset=['CodItem'])
print(df_i3.shape)
df_i3[(df_i3.Grupo2 == "DESCONOCIDO") | (df_i3.Grupo3 == "DESCONOCIDO")]

(48365, 6)


,Grupo2,Grupo3,CodGrupo4,Grupo4,CodItem,Descripcion
226,DESCONOCIDO,ABYA - YALA,L04,Educación - Sociología,L043068,BUEN VIVIR SUMAK KAWSAY
248,DESCONOCIDO,ABYA - YALA,L04,Educación - Sociología,L043081,TRANSFORMACION ANTROPOLOGICA DEL SIGLO XXI
278,VARIOS AUTORES,DESCONOCIDO,L04,Educación - Sociología,L043102,"CORREISMO AL DESNUDO, EL"
940,DESCONOCIDO,LIBSA,L04,Educación - Sociología,L043549,TESTS DE MENSA PARA EJERCITAR TU CEREBRO
1004,"ACOSTA, ALBERTO",DESCONOCIDO,L04,Educación - Sociología,L043585,"RESTAURACION CONSERVADORA DEL CORREISMO, LA"
...,...,...,...,...,...,...
79090,"QUIZHPE PERALTA, ARTURO",DESCONOCIDO,L04,Educación - Sociología,L042357,MINKA POR LA VIDA encuentro continental de los...
79339,RUILOVA- EDUARDO,DESCONOCIDO,L04,Educación - Sociología,L042511,ENTRE EL CONSENSO DE BUENOS AIRES Y EL SOCIALI...
79505,"LARA, JAIME",DESCONOCIDO,L04,Educación - Sociología,L042606,JUEGOS DE OTROS TIEMPOS T1
79509,DESCONOCIDO,TURNER,L04,Educación - Sociología,L042609,POST IT CITY ciudades ocasionales-cidades ocas...


Como vemos, se han eliminado 31629 tuplas de los productos que estaban repetidos dentro del dataset, quedando 48365 filas válidas que aportan información en el dataset de inventario.

## 3. Transformacion de datos
#### 3.1. Integración de datos:
Para obtener un solo dataframe (df) que contenga todos los datos, unimos ambos datasets tomando como clave de unión el código de cada ítem

In [13]:
df = df_v2.merge(df_i3, on='CodItem')

In [14]:
print(df.shape)
df.head(100)

(83202, 16)


,NombreVend,Desc.Línea,CodItem,Descripcion_x,Trans,Cod.Tipo Cliente,Fecha,Cod.Modelo,Cantidad,PrecioT,Utilidad,Grupo2,Grupo3,CodGrupo4,Grupo4,Descripcion_y
0,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ...",FC 21353,1.0,27/03/2015,P11,-1,51.00,36.05,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
1,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ...",FC 20398,1.0,31/01/2015,P11,-1,51.00,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
2,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ...",FC 20399,1.0,31/01/2015,P11,-1,51.00,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
3,ESPERANZA GALARZA,ENCICLOPEDIAS,P110170,"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ...",FC 31701,1.0,21/12/2016,P11,-1,37.70,22.55,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
4,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ...",FC 20402,1.0,31/01/2015,P11,-1,51.00,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,ESPERANZA GALARZA,ENCICLOPEDIAS,P190078,LO MEJOR DE LA COCINA ECUATORIANA (BRAL´S),FC 39416,1.0,11/06/2018,P19,-1,24.05,13.05,VARIOS AUTORES,BRAL'S,P19,Cocina - Bebidas y Repostería,LO MEJOR DE LA COCINA ECUATORIANA (BRAL´S)
96,ESPERANZA GALARZA,ENCICLOPEDIAS,P190078,LO MEJOR DE LA COCINA ECUATORIANA (BRAL´S),FC 38272,1.0,08/03/2018,P19,-1,24.05,10.64,VARIOS AUTORES,BRAL'S,P19,Cocina - Bebidas y Repostería,LO MEJOR DE LA COCINA ECUATORIANA (BRAL´S)
97,LOURDES SEVILLA,ENCICLOPEDIAS,P190078,LO MEJOR DE LA COCINA ECUATORIANA (BRAL´S),FCUDA 7500,1.0,27/04/2018,P19,-1,24.05,13.05,VARIOS AUTORES,BRAL'S,P19,Cocina - Bebidas y Repostería,LO MEJOR DE LA COCINA ECUATORIANA (BRAL´S)
98,GLENDA SANCHEZ,ENCICLOPEDIAS,P190078,LO MEJOR DE LA COCINA ECUATORIANA (BRAL´S),FC 39184,1.0,21/05/2018,P19,-1,24.05,13.05,VARIOS AUTORES,BRAL'S,P19,Cocina - Bebidas y Repostería,LO MEJOR DE LA COCINA ECUATORIANA (BRAL´S)


### 3.2. Suavizar datos 
Se puede observar que al unir los dataframes tenemos columnas con datos iguales: 
- CodModelo y CodGrupo4
- Descripcion_x y Descripcion_y
Por lo que procedemos a borrar la primera de cada par de columnas iguales

In [15]:
df=df.drop(['Cod.Modelo','Descripcion_x'], axis=1)


In [16]:
df.head(-100)

,NombreVend,Desc.Línea,CodItem,Trans,Cod.Tipo Cliente,Fecha,Cantidad,PrecioT,Utilidad,Grupo2,Grupo3,CodGrupo4,Grupo4,Descripcion_y
0,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 21353,1.0,27/03/2015,-1,51.00,36.05,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
1,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 20398,1.0,31/01/2015,-1,51.00,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
2,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 20399,1.0,31/01/2015,-1,51.00,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
3,ESPERANZA GALARZA,ENCICLOPEDIAS,P110170,FC 31701,1.0,21/12/2016,-1,37.70,22.55,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
4,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 20402,1.0,31/01/2015,-1,51.00,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83097,GLENDA SANCHEZ,LIBRERIA,L112983,FC 19952,1.0,05/01/2015,-1,7.14,2.14,DESCONOCIDO,DESCONOCIDO,L11,Infantil - Juvenil,LEANING TOWER Puzzle 3D
83098,LUCIA IZQUIERDO,LIBRERIA,L071533,FC 19969,1.0,06/01/2015,-1,16.00,4.80,"BENEDETTI, MARIO",ALIANZA,L07,Narrativa clásica y contemporánea - Literatura,"TREGUA, LA"
83099,LOURDES SEVILLA,LIBRERIA,L100532,FE 4039,1.0,03/10/2019,-1,22.00,6.60,"ASIMOV, ISAAC",ALIANZA,L10,Interés general - Estudiantiles,"CERCANO ORIENTE, EL historia universal"
83100,LOURDES SEVILLA,LIBRERIA,L101279,FE 4039,1.0,03/10/2019,-1,25.95,9.08,"WEITZ, ERIC",TURNER,L10,Interés general - Estudiantiles,"ALEMANIA DE WEIMAR, LA Presagio y tragedia"


### 3.3. Detección y resolución de conflictos de valores de datos
- La cantidad de artículos comprados es un número negativo, por lo que se obtendrá el valor absoluto para obtener valores positivos

In [17]:
df['Cantidad']=np.abs(df['Cantidad'])
df.head()

,NombreVend,Desc.Línea,CodItem,Trans,Cod.Tipo Cliente,Fecha,Cantidad,PrecioT,Utilidad,Grupo2,Grupo3,CodGrupo4,Grupo4,Descripcion_y
0,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 21353,1.0,27/03/2015,1,51.0,36.05,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
1,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 20398,1.0,31/01/2015,1,51.0,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
2,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 20399,1.0,31/01/2015,1,51.0,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
3,ESPERANZA GALARZA,ENCICLOPEDIAS,P110170,FC 31701,1.0,21/12/2016,1,37.7,22.55,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
4,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 20402,1.0,31/01/2015,1,51.0,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,Infantil - Juvenil (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."


- Como se visualiza, tenemos varias variables de tipo Object, por lo que la Fecha la transfomaremos a tipo Date y Desc.Línea a String para utilizarlos posteriormente para agregar columnas basadas en estos datos. De igual manera, se transforma a int la columna Tipo Cliente

In [18]:
print(df.dtypes)

NombreVend           object
Desc.Línea           object
CodItem              string
Trans                object
Cod.Tipo Cliente    float64
Fecha                object
Cantidad              int64
PrecioT             float64
Utilidad            float64
Grupo2               object
Grupo3               object
CodGrupo4            object
Grupo4               object
Descripcion_y        object
dtype: object


In [19]:
df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Desc.Línea']=df['Desc.Línea'].astype('string')
df['Grupo4']=df['Grupo4'].astype('string')
df['Cod.Tipo Cliente']=df['Cod.Tipo Cliente'].astype(int)
print(df.dtypes)

NombreVend                  object
Desc.Línea                  string
CodItem                     string
Trans                       object
Cod.Tipo Cliente             int64
Fecha               datetime64[ns]
Cantidad                     int64
PrecioT                    float64
Utilidad                   float64
Grupo2                      object
Grupo3                      object
CodGrupo4                   object
Grupo4                      string
Descripcion_y               object
dtype: object


- Adicionalmente, vamos a renombrar ciertas columnas  ya que sus nombres no son tan descriptivos para saber que información tiene la columna:
    - NombreVend -> Vendedor
    - Desc.Línea -> Grupo
    - Cod.Tipo Cliente -> TipoCliente
    - PrecioT -> PrecioTotal
    - Grupo2 -> Autor
    - Grupo3 -> Editorial
    - CodGrupo4 -> CodCategoria
    - Grupo4 -> Categoria
    - Descripcion_y -> NombreLibro

In [20]:
df = df.rename(columns={'NombreVend':'Vendedor',
                                   'Desc.Línea':'Grupo',
                                   'Cod.Tipo Cliente':'TipoCliente',
                                   'PrecioT':'PrecioTotal',
                                   'Grupo2':'Autor',
                                   'Grupo3':'Editorial',
                                   'CodGrupo4':'CodCategoria',
                                   'Grupo4':'Categoria',
                                   'Descripcion_y':'NombreLibro'})
df.columns

Index(['Vendedor', 'Grupo', 'CodItem', 'Trans', 'TipoCliente', 'Fecha',
       'Cantidad', 'PrecioTotal', 'Utilidad', 'Autor', 'Editorial',
       'CodCategoria', 'Categoria', 'NombreLibro'],
      dtype='object')

- Otro de los problemas que se puede llegar a tener  es que  los autores, las editoriales o los nombres de categoría se hayan escrito de diferentes manera. Por ejemplo para la editorial podríamos tener LEXUS, Lexus, lexus, etc. Esto implicaría que se crearían tres grupos para la misma editorial. Para ello transformaremos toda la información de estas tres columnas a mayúsculas.

In [21]:
df['Autor'] = df['Autor'].str.upper()
df['Editorial'] = df['Editorial'].str.upper()
df['Categoria'] = df['Categoria'].str.upper()
df.head(-100)

,Vendedor,Grupo,CodItem,Trans,TipoCliente,Fecha,Cantidad,PrecioTotal,Utilidad,Autor,Editorial,CodCategoria,Categoria,NombreLibro
0,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 21353,1,2015-03-27,1,51.00,36.05,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,INFANTIL - JUVENIL (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
1,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 20398,1,2015-01-31,1,51.00,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,INFANTIL - JUVENIL (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
2,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 20399,1,2015-01-31,1,51.00,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,INFANTIL - JUVENIL (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
3,ESPERANZA GALARZA,ENCICLOPEDIAS,P110170,FC 31701,1,2016-12-21,1,37.70,22.55,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,INFANTIL - JUVENIL (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
4,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 20402,1,2015-01-31,1,51.00,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,INFANTIL - JUVENIL (P),"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83097,GLENDA SANCHEZ,LIBRERIA,L112983,FC 19952,1,2015-05-01,1,7.14,2.14,DESCONOCIDO,DESCONOCIDO,L11,INFANTIL - JUVENIL,LEANING TOWER Puzzle 3D
83098,LUCIA IZQUIERDO,LIBRERIA,L071533,FC 19969,1,2015-06-01,1,16.00,4.80,"BENEDETTI, MARIO",ALIANZA,L07,NARRATIVA CLÁSICA Y CONTEMPORÁNEA - LITERATURA,"TREGUA, LA"
83099,LOURDES SEVILLA,LIBRERIA,L100532,FE 4039,1,2019-03-10,1,22.00,6.60,"ASIMOV, ISAAC",ALIANZA,L10,INTERÉS GENERAL - ESTUDIANTILES,"CERCANO ORIENTE, EL historia universal"
83100,LOURDES SEVILLA,LIBRERIA,L101279,FE 4039,1,2019-03-10,1,25.95,9.08,"WEITZ, ERIC",TURNER,L10,INTERÉS GENERAL - ESTUDIANTILES,"ALEMANIA DE WEIMAR, LA Presagio y tragedia"


- Otro de los problemas que se tienen, es que existen categorías que son las mismas pero que se han escrito de forma diferente, por ejemplo se tiene "COCINA - BEBIDAS - RESPOSTERÍA" y por otro lado "COCINA - BEBIDAS - REPOSTERÍA", por lo que tendremos que modificar para que esto no genere  inconvenientes.

In [22]:
print(sorted(df['Categoria'].unique()))

['AGROPECUARIA - CCNN - ECOLOGÍA - MEDIO AMBIENTE', 'AGROPECUARIA - CCNN - ECOLOGÍA Y MEDIO AMBIENTE', 'ARTE - DECORACIÓN - DISEÑO', 'ARTE - DISEÑO - DECORACIÓN', 'AUTOAYUDA - FAMILIA - SEXUALIDAD', 'AUTOAYUDA - ORIENTACIÓN FAMILIAR', 'BELLEZA - MANUALIDADES - HOGAR', 'BELLEZA - MANUALIDADES Y HOGAR', 'CIENCIAS EXACTAS', 'CIENCIAS EXACTAS (P)', 'COCINA - BEBIDAS - REPOSTERÍA', 'COCINA - BEBIDAS Y REPOSTERÍA', 'DEPORTES', 'DEPORTES (P)', 'DERECHO', 'DERECHO (P)', 'DIVULGACIÓN CIENTÍFICA Y TECNOLOGÍA', 'EDUCACIÓN - PSICOLOGÍA - PEDAGOGÍA', 'EDUCACIÓN - SOCIOLOGÍA', 'EMPRESARIALES', 'EMPRESARIALES (P)', 'ESOTERISMO - MISTERIO', 'ESPIRITUALIDAD - FILOSOFÍA', 'ESPIRITUALIDAD - FILOSOFÍA (P)', 'HUMOR - HISTORIETAS - TIEMPO LIBRE', 'INFANTIL - JUVENIL', 'INFANTIL - JUVENIL (P)', 'INFORMÁTICA Y COMPUTACIÓN', 'INFORMÁTICA Y COMPUTACIÓN (P)', 'INTERÉS GENERAL - ESTUDIANTILES', 'INTERÉS GENERAL - ESTUDIANTILES (P)', 'LENGUAJE - IDIOMAS - DICCIONARIOS', 'LENGUAJE - IDIOMAS - DICCIONARIOS (P)', 'LI

In [23]:
df['Categoria'] = df['Categoria'].map({'AGROPECUARIA - CCNN - ECOLOGÍA Y MEDIO AMBIENTE':'AGROPECUARIA - CCNN - ECOLOGÍA - MEDIO AMBIENTE',
                            'AGROPECUARIA - CCNN - ECOLOGÍA - MEDIO AMBIENTE':'AGROPECUARIA - CCNN - ECOLOGÍA - MEDIO AMBIENTE',
                             'ARTE - DECORACIÓN - DISEÑO':'ARTE - DISEÑO - DECORACIÓN',
                             'ARTE - DISEÑO - DECORACIÓN':'ARTE - DISEÑO - DECORACIÓN',
                             'AUTOAYUDA - FAMILIA - SEXUALIDAD':'AUTOAYUDA - ORIENTACIÓN FAMILIAR',
                             'AUTOAYUDA - ORIENTACIÓN FAMILIAR':'AUTOAYUDA - ORIENTACIÓN FAMILIAR',
                            'BELLEZA - MANUALIDADES Y HOGAR':'BELLEZA - MANUALIDADES - HOGAR',
                            'BELLEZA - MANUALIDADES - HOGAR':'BELLEZA - MANUALIDADES - HOGAR',
                            'CIENCIAS EXACTAS (P)':'CIENCIAS EXACTAS',
                            'CIENCIAS EXACTAS':'CIENCIAS EXACTAS',
                            'COCINA - BEBIDAS Y REPOSTERÍA':'COCINA - BEBIDAS - REPOSTERÍA',
                            'COCINA - BEBIDAS - REPOSTERÍA':'COCINA - BEBIDAS - REPOSTERÍA',
                            'DEPORTES (P)':'DEPORTES',
                            'DEPORTES':'DEPORTES',
                            'DERECHO (P)':'DERECHO',
                            'DERECHO':'DERECHO',
                            'ESOTERISMO - MISTERIO':'ESOTERISMO - MISTERIO',
                            'HUMOR - HISTORIETAS - TIEMPO LIBRE':'HUMOR - HISTORIETAS - TIEMPO LIBRE',
                            'INTERÉS GENERAL - ESTUDIANTILES (P)':'INTERÉS GENERAL - ESTUDIANTILES',
                            'INTERÉS GENERAL - ESTUDIANTILES':'INTERÉS GENERAL - ESTUDIANTILES',
                            'LIBROS REGALO':'LIBROS REGALO',
                            'OBRAS DE LA CASA DE LA CULTURA':'OBRAS DE LA CASA DE LA CULTURA',
                            'OFERTAS - PROMOCIONES':'OFERTAS - PROMOCIONES',
                            'REVISTAS Y AFINES':'REVISTAS Y AFINES',
                            'TURISMO Y AFINES':'TURISMO Y AFINES',
                            'DIVULGACIÓN CIENTÍFICA Y TECNOLOGÍA':'DIVULGACIÓN CIENTÍFICA Y TECNOLOGÍA',
                            'EDUCACIÓN - SOCIOLOGÍA':'EDUCACIÓN - PSICOLOGÍA - PEDAGOGÍA',
                            'EDUCACIÓN - PSICOLOGÍA - PEDAGOGÍA':'EDUCACIÓN - PSICOLOGÍA - PEDAGOGÍA',
                            'EMPRESARIALES (P)':'EMPRESARIALES',
                            'EMPRESARIALES':'EMPRESARIALES',
                            'ESPIRITUALIDAD - FILOSOFÍA (P)':'ESPIRITUALIDAD - FILOSOFÍA',
                            'ESPIRITUALIDAD - FILOSOFÍA':'ESPIRITUALIDAD - FILOSOFÍA',
                            'INFANTIL - JUVENIL (P)':'INFANTIL - JUVENIL',
                            'INFANTIL - JUVENIL':'INFANTIL - JUVENIL',
                            'INFORMÁTICA Y COMPUTACIÓN (P)':'INFORMÁTICA Y COMPUTACIÓN',
                            'INFORMÁTICA Y COMPUTACIÓN':'INFORMÁTICA Y COMPUTACIÓN',
                            'INTERÉS GENERAL - ESTUDIANTILES (P)':'INTERÉS GENERAL - ESTUDIANTILES',
                            'INTERÉS GENERAL - ESTUDIANTILES':'INTERÉS GENERAL - ESTUDIANTILES',
                            'LENGUAJE - IDIOMAS - DICCIONARIOS (P)':'LENGUAJE - IDIOMAS - DICCIONARIOS',
                            'LENGUAJE - IDIOMAS - DICCIONARIOS':'LENGUAJE - IDIOMAS - DICCIONARIOS',
                            'LIBROS TÉCNICOS (P)':'LIBROS TÉCNICOS',
                            'LIBROS TÉCNICOS':'LIBROS TÉCNICOS',
                            'MEDICINA Y SALUD (P)':'MEDICINA Y SALUD',
                            'MEDICINA Y SALUD':'MEDICINA Y SALUD',
                            'LITERATURA':'NARRATIVA CLÁSICA Y CONTEMPORÁNEA - LITERATURA',
                            'NARRATIVA CLÁSICA Y CONTEMPORÁNEA - LITERATURA':'NARRATIVA CLÁSICA Y CONTEMPORÁNEA - LITERATURA',
                            'TESTIMONIO - INVESTIGACIÓN - COMUNICACIÓN':'TESTIMONIO-COMUNICACIÓN- INVESTIGACIÓN -BIOGRAFÍAS'},na_action='ignore')

#df3['Categoria'].value_counts(dropna=False)
print(df['Categoria'].unique())

['INFANTIL - JUVENIL' 'LIBROS TÉCNICOS' 'COCINA - BEBIDAS - REPOSTERÍA'
 'MEDICINA Y SALUD' 'INTERÉS GENERAL - ESTUDIANTILES'
 'LENGUAJE - IDIOMAS - DICCIONARIOS'
 'AGROPECUARIA - CCNN - ECOLOGÍA - MEDIO AMBIENTE'
 'EDUCACIÓN - PSICOLOGÍA - PEDAGOGÍA' 'ESOTERISMO - MISTERIO'
 'AUTOAYUDA - ORIENTACIÓN FAMILIAR' 'BELLEZA - MANUALIDADES - HOGAR'
 'ARTE - DISEÑO - DECORACIÓN' 'ESPIRITUALIDAD - FILOSOFÍA'
 'CIENCIAS EXACTAS' 'NARRATIVA CLÁSICA Y CONTEMPORÁNEA - LITERATURA'
 'DERECHO' 'EMPRESARIALES' 'INFORMÁTICA Y COMPUTACIÓN' 'DEPORTES'
 'LIBROS REGALO' 'HUMOR - HISTORIETAS - TIEMPO LIBRE' 'TURISMO Y AFINES'
 'DIVULGACIÓN CIENTÍFICA Y TECNOLOGÍA'
 'TESTIMONIO-COMUNICACIÓN- INVESTIGACIÓN -BIOGRAFÍAS' 'REVISTAS Y AFINES'
 nan 'OFERTAS - PROMOCIONES' 'OBRAS DE LA CASA DE LA CULTURA']


### 3.3. Construcción de atributos

Se van a agregar tres columnas derivadas de los datos existentes:
- CodGrupo: Grupo es una variable categórica que puede tomar los valores de ENCICLOPEDIAS o LIBRERIA, se creará  esta nueva variable  asignando el valor de 0 si es ENCICLOPEDIAS o 1 si es LIBRERIA. 
- CuartilAño: A partir de la fecha de compra se establecerán los valores Q1, Q2, Q3 o Q4 para saber en que cuartil del año se realizó la compra.
- Año: A partir de la fecha de compra se obtendrá el año de cada compra para realizar proyecciones posteriores.

In [24]:
df["CodGrupo"] = df.apply(lambda x: 0 if x['Grupo'] == "ENCICLOPEDIAS" else 1, axis=1)
conditionlist = [
    df["Fecha"].dt.month<=3,
    df["Fecha"].dt.month<=6,
    df["Fecha"].dt.month<=9,
    df["Fecha"].dt.month>=10]
choicelist = ['Q1', 'Q2', 'Q3','Q4']
df['CuartilAño'] = np.select(conditionlist, choicelist,default='Q1')
df["Año"] = df["Fecha"].dt.year
df.head(-1)

,Vendedor,Grupo,CodItem,Trans,TipoCliente,Fecha,Cantidad,PrecioTotal,Utilidad,Autor,Editorial,CodCategoria,Categoria,NombreLibro,CodGrupo,CuartilAño,Año
0,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 21353,1,2015-03-27,1,51.00,36.05,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,INFANTIL - JUVENIL,"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ...",0,Q1,2015
1,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 20398,1,2015-01-31,1,51.00,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,INFANTIL - JUVENIL,"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ...",0,Q1,2015
2,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 20399,1,2015-01-31,1,51.00,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,INFANTIL - JUVENIL,"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ...",0,Q1,2015
3,ESPERANZA GALARZA,ENCICLOPEDIAS,P110170,FC 31701,1,2016-12-21,1,37.70,22.55,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,INFANTIL - JUVENIL,"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ...",0,Q4,2016
4,LOURDES SEVILLA,ENCICLOPEDIAS,P110170,FC 20402,1,2015-01-31,1,51.00,36.26,"CADAVID SANMIGUEL, LILIANA",LEXUS,P11,INFANTIL - JUVENIL,"GRAN LIBRO DE LOS VALORES, EL (LEXUS) cuentos ...",0,Q1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83196,LUIS PERALTA Q.,LIBRERIA,L072298,FC 21445,1,2015-02-04,1,4.00,1.71,MOLIERE,ALBA,L07,NARRATIVA CLÁSICA Y CONTEMPORÁNEA - LITERATURA,"ENFERMO IMAGINARIO, EL",1,Q1,2015
83197,GLENDA SANCHEZ,LIBRERIA,L141031,FC 21465,1,2015-04-04,1,52.00,13.30,"HUIDOBRO MOYA, JOSE MANUEL",PARANINFO,L14,LIBROS TÉCNICOS,SISTEMAS TELEMATICOS,1,Q2,2015
83198,CLAUDIA BRAVO O.,LIBRERIA,L190115,FC 21547,0,2015-08-04,1,3.50,1.23,DESCONOCIDO,TRIDENT,L19,COCINA - BEBIDAS - REPOSTERÍA,PIZZAS IRRESISTIBLES Chef Express,1,Q3,2015
83199,LUCIA IZQUIERDO,LIBRERIA,L190647,FC 21557,1,2015-09-04,1,6.75,2.36,VARIOS AUTORES,TRIDENT,L19,COCINA - BEBIDAS - REPOSTERÍA,BARBACOA PARA SORPRENDER COCINA PRACTICA,1,Q3,2015


In [25]:
print(df.shape)

(83202, 17)


In [26]:
df.to_excel("data_salida.xlsx")

## 4. Conclusiones

Luego de realizar el proceso de limpieza y transformación de los datos, es muy interesante de ver que se ha obtenido un solo dataset con información relevante para continuar con el objetivo planteado.

Se partió de dos dataset bastante grandes, y se redujo a uno solo pero con datos en formatos correctos, sin datos basura o repetidos.

El proceso de limpieza y transformación, realmente es un paso importante al momento dentro del proceso KDD.